In [38]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
#Author: Selvaria
#整合成品油指数，自定义计算

import pandas
import pymysql
import numpy as np
import datetime

class IndexSelf(object): #目前都是0#柴油价格
    
    def __init__(self, date):
        self.date = date
        #pass
        
    def market_data(self): #全国市场平均价
        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
        sql = "SELECT `type`, `DATE`,avg(`price`) FROM `out_price_market_value` v JOIN `out_price_market_key` k \
        ON k.`id` = v.`ref_key_id` WHERE `source_id` = '3' AND k.`type` LIKE '0#%' and v.`price` > 100 group BY date"
        df_excel_market = pandas.read_sql(sql, connection)
        #print(df_excel_market.shape)
        df_m = df_excel_market[['DATE','avg(`price`)']]
        return df_m
    
    def factory_data(self): #全国平均出厂价
        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
        sql = "SELECT `type`, `DATE`,avg(`price`), `factory`, `factory_type` FROM `out_price_factory_value` v JOIN `out_price_factory_key` k \
        ON k.`id` = v.`ref_key_id` where `source_id` = '3' AND k.`type` LIKE '0#%' and v.`price` > 100 group BY date"
        df_excel_factory = pandas.read_sql(sql, connection)
        df_f = df_excel_factory[['DATE','avg(`price`)']]
        return df_f
    
    def hcb_data(self): #货车帮每日均价
        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
        sql = "SELECT `type`, `factory`, AVG(`price`), v.`date` FROM `hcb_factory_price_value` v JOIN `hcb_factory_price_key` k \
        ON k.`id` = v.`ref_key_id` WHERE k.`type` LIKE '0#%' group BY date"
        df_excel_hcb = pandas.read_sql(sql, connection)
        df_h = df_excel_hcb[['date','AVG(`price`)']]
        return df_h
    
    def handle_df(self): #处理整合到一个表
        df_market = self.market_data()
        df_factory = self.factory_data()
        df_hcb = self.hcb_data()
        
        df_market.rename(columns={'avg(`price`)':'price_market'}, inplace = True)
        df_factory.rename(columns={'avg(`price`)':'price_factory'}, inplace = True)
        df_hcb.columns = ['DATE','price_hcb'] 
        
        df_merge1 = pandas.merge(df_factory, df_market, how='left', on='DATE')
        df_merge1['DATE'] = df_merge1['DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))
        df_merge1['loss_market'] = df_merge1['price_market'].isna()
        
#         df_real = pandas.read_excel(r'D:\data/东北炼厂报价.xls') #每日实际报价在这，正式运行后具体的传入方式待定
#         df_real = df_real[['日期','价格','产品','炼厂']]
#         df_s = df_real[(df_real['产品']=='国六0#')|(df_real['产品']=='国五0#')]
#         df_s['日期'] = df_s['日期'].apply(lambda x: x.strftime('%Y-%m-%d'))
#         df_handle = df_s[df_s['炼厂']!='缘泰石油']
#         df_handle = df_handle.groupby("日期").mean().reset_index()

        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
        sql_r = "SELECT `name`,`date`,avg(`price`), `product`, `position`,`rank` FROM `northeast_local_price` where `name` != '缘泰石油' \
        and `product` LIKE '%0#' group BY `date`"
        df_real = pandas.read_sql(sql_r, connection) #每日实际报价在这，正式运行后具体的传入方式待定
        df_s = df_real[['date','avg(`price`)']]
        df_s['date'] = df_s['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
        df_handle = df_s.copy()
        df_handle.columns = ['DATE','price_real'] 
        df_merge2 = pandas.merge(df_merge1, df_handle, how='left', on='DATE')
        
        df_merge3 = pandas.merge(df_merge1, df_handle, how='right', on='DATE')
        df_plus = df_merge2.append(df_merge3, ignore_index=True)
        df_minus = df_plus.drop_duplicates(subset=['DATE'],keep='last')
        df_minus['loss_real'] = df_minus['price_real'].isna()      
        df_final = df_minus.sort_values('DATE').reset_index(drop=True)
        
        df_final_h = pandas.merge(df_final, df_hcb, how='left', on='DATE')
        df_final_h['loss_hcb'] = df_final_h['price_hcb'].isna()
        df_need = df_final_h[(df_final_h['loss_market']==False)|(df_final_h['loss_real']==False)|(df_final_h['loss_hcb']==False)]
        df_need.reset_index(drop=True, inplace=True)
        return df_need
    
    def single_data(self, s_date): #返回单行数据
        df_need = self.handle_df()
        df_single = df_need[df_need['DATE']==s_date].reset_index()
        if len(df_single) < 1:
            date_new = datetime.datetime.strptime(date, '%Y-%m-%d') - datetime.timedelta(days=1)
            date_new_str = date_new.strftime('%Y-%m-%d')
            return self.single_data(s_date=date_new_str)
        else:
            data_row = list(df_single.values[0])[1:]
            data_row[2] = self.recursive_date(data_row, 3, s_date)
            #print(data_row)
            data_row[4] = self.recursive_date(data_row, 5, s_date)
            #print(data_row)
            data_row[6] = self.recursive_date(data_row, 7, s_date)
            #print(data_row)
            return data_row
        
    def recursive_date(self, data, code, r_date): #递归返回有日期的数据
        df_need = self.handle_df()
        df_single = df_need[df_need['DATE']==r_date].reset_index()
        if len(df_single) < 1 or data[code] == True:
            date_new = datetime.datetime.strptime(r_date, '%Y-%m-%d') - datetime.timedelta(days=1)
            date_new_str = date_new.strftime('%Y-%m-%d')
            #print(date_new_str)
            df_single = df_need[df_need['DATE']==date_new_str].reset_index()
            try:
                data_row = list(df_single.values[0])[1:]
            except Exception as e:
                print(str(e))
                data_row = data
                pass
            return self.recursive_date(data_row, code, r_date=date_new_str)
        else:
            return data[code-1]
        
    def output_data(self): #['2019-08-05', 6751.653992395437, 6877.241379310345, False, 6590.0, False, 6556.775, False]
        #out_dict = {'相对公开出厂价':np.nan, '相对公开市场价':np.nan, '相对全国实际平均价':np.nan, '油价平均指数':np.nan}
        out_dict = {'rela_factory':np.nan, 'rela_market':np.nan, 'rela_avg':np.nan, 'avg_market':np.nan}
        data_row = self.single_data(self.date)
        print(data_row)
        if data_row == None:
            print('本日无数据')
            return out_dict
#         elif data_row[3] == False and data_row[5] == False and data_row[7] == False:
#             out_dict['相对公开出厂价'] = (data_row[4] - data_row[1])/data_row[1] #实际报价相对卓创出厂价的增减
#             out_dict['相对公开市场价'] = (data_row[4] - data_row[2])/data_row[2] #实际报价相对卓创市场价的增减
#             out_dict['相对全国实际平均价'] = (data_row[4] - data_row[6])/data_row[6] #实际报价相对货车帮平均价的增减
#             out_dict['油价平均指数'] = (data_row[6] - data_row[2])/data_row[2] #货车帮平均价相对卓创市场价的增减
#             return out_dict
#         elif data_row[3] == True and data_row[5] == False and data_row[7] == False:
#             out_dict['相对公开出厂价'] = (data_row[4] - data_row[1])/data_row[1]
#             #out_dict['相对公开市场价'] = (data_row[4] - data_row[2])/data_row[2]
#             out_dict['相对全国实际平均价'] = (data_row[4] - data_row[6])/data_row[6] 
#             return out_dict
#         elif data_row[3] == False and data_row[5] == False and data_row[7] == True:
#             out_dict['相对公开出厂价'] = (data_row[4] - data_row[1])/data_row[1]
#             out_dict['相对公开市场价'] = (data_row[4] - data_row[2])/data_row[2]
#             #out_dict['相对全国实际平均价'] = (data_row[4] - data_row[6])/data_row[6]
#             return out_dict
#         elif data_row[3] == True and data_row[5] == False and data_row[7] == True:
#             out_dict['相对公开出厂价'] = (data_row[4] - data_row[1])/data_row[1]
#             #out_dict['相对公开市场价'] = (data_row[4] - data_row[2])/data_row[2]
#             #out_dict['相对全国实际平均价'] = (data_row[4] - data_row[6])/data_row[6]
#         elif data_row[3] == False and data_row[5] == True and data_row[7] == False:
#             out_dict['油价平均指数'] = (data_row[6] - data_row[2])/data_row[2]
#             return out_dict
        else:
#             print('部分数据缺失，无法计算')
#             out_dict['相对公开出厂价'] = (data_row[4] - data_row[1])/data_row[1] #实际报价相对卓创出厂价的增减
#             out_dict['相对公开市场价'] = (data_row[4] - data_row[2])/data_row[2] #实际报价相对卓创市场价的增减
#             out_dict['相对全国实际平均价'] = (data_row[4] - data_row[6])/data_row[6] #实际报价相对货车帮平均价的增减
#             out_dict['油价平均指数'] = (data_row[6] - data_row[2])/data_row[2] #货车帮平均价相对卓创市场价的增减
            out_dict['rela_factory'] = (data_row[4] - data_row[1])/data_row[1] 
            out_dict['rela_market'] = (data_row[4] - data_row[2])/data_row[2]
            out_dict['rela_avg'] = (data_row[4] - data_row[6])/data_row[6] 
            out_dict['avg_market'] = (data_row[6] - data_row[2])/data_row[2] 
            return out_dict

#ois = IndexSelf('2019-08-29')
#ois = IndexSelf('2019-08-05')
#ois = IndexSelf('2019-08-14')
#ois = IndexSelf('2019-01-03')
#ois = IndexSelf('2019-08-08')

now = datetime.datetime.now()
today = now.strftime('%Y-%m-%d')

def main(date = today):
    ois = IndexSelf(date)
    #df= ois.handle_df()
    #df.head()
    json_data = ois.output_data()
    return json_data    

main()


D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
['2019-09-03', 6653.764258555133, 6877.241379310345, True, 6465.0, True, 6518.333333333333, False]


{'avg_market': -0.0521877924856265,
 'rela_avg': -0.008182050626438205,
 'rela_factory': -0.028369544098654796,
 'rela_market': -0.05994283995186526}

In [33]:
df_real = pandas.read_excel(r'D:\data/东北炼厂报价.xls') #每日实际报价在这，正式运行后具体的传入方式待定
df_real = df_real[['炼厂','日期','价格','产品','省市','当日排序']]
df_real['日期'] = df_real['日期'].apply(lambda x: x.strftime('%Y-%m-%d'))
connection_f = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
for index in df_real.index:
    data_row = list(df_real.loc[index].values)
    data = [str(x) for x in data_row]
    with connection_f.cursor() as cursor:
        sql_k = "insert into `northeast_local_price`(`name`,`date`,`price`, `product`, `position`,`rank`) values(%s,%s,%s,%s,%s,%s)"
        cursor.execute(sql_k,data)
    connection_f.commit()
connection_f.close()

print('Done')

D:\Selvaria\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect integer value: 'nan' for column 'rank' at row 1")
  result = self._query(query)
D:\Selvaria\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect integer value: 'nan' for column 'price' at row 1")
  result = self._query(query)


Done
